In [1]:
import numpy as np
import math as m
import scipy.linalg as linalg
from scipy import stats
from timeit import default_timer

Example from:

James Ma Weiming -- Mastering Python for Finance https://www.packtpub.com/big-data-and-business-intelligence/mastering-python-finance

(see: https://github.com/jamesmawm/Mastering-Python-for-Finance-source-codes/tree/master/B03898_04_codes)

In [2]:
#""" Shared attributes and functions of FD """ 
class FiniteDifferences(object):
    def __init__(self, S0, K, r, T, sigma, Smax, M, N,
                 is_call=True):
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.sigma = sigma
        self.Smax = Smax
        self.M, self.N = int(M), int(N)  # Ensure M & N are integers
        self.is_call = is_call
        self.dS = Smax / float(self.M)
        self.dt = T / float(self.N)
        self.i_values = np.arange(self.M)
        self.j_values = np.arange(self.N)
        self.grid = np.zeros(shape=(self.M+1, self.N+1))
        self.boundary_conds = np.linspace(0, Smax, self.M+1)
        
    def _setup_boundary_conditions_(self):
        if self.is_call:
            self.grid[:, -1] = np.maximum(
                self.boundary_conds - self.K, 0)
            self.grid[-1, :-1] = (self.Smax - self.K) * \
                                 np.exp(-self.r *
                                        self.dt *
                                        (self.N-self.j_values))
        else:
            self.grid[:, -1] = \
                np.maximum(self.K-self.boundary_conds, 0)
            
            self.grid[0, :-1] = (self.K - self.Smax) * \
                np.exp(-self.r *
                       self.dt *
                       (self.N-self.j_values))
    
    def _setup_coefficients_(self):
        pass
    
    def _traverse_grid_(self):
        """
        Iterate the grid backwards in time 
        """
        pass
    
    def _interpolate_(self):
        """
        Use piecewise linear interpolation on the initial
        grid column to get the closest price at S0.
        """
        return np.interp(self.S0,
                         self.boundary_conds,
                         self.grid[:, 0]) 
    
    def price(self):
        self._setup_boundary_conditions_()
        self._setup_coefficients_()
        self._traverse_grid_()
        return self._interpolate_()

In [3]:
class FDExplicitEu(FiniteDifferences):    
    def _setup_coefficients_(self):
        self.a = 0.5*self.dt*((self.sigma**2) *
                              (self.i_values**2) -
                              self.r*self.i_values)
        
        self.b = 1 - self.dt*((self.sigma**2) * 
                              (self.i_values**2) +
                              self.r)
        
        self.c = 0.5*self.dt*((self.sigma**2) *
                              (self.i_values**2) +
                              self.r*self.i_values)
    
    def _traverse_grid_(self):
        for j in reversed(self.j_values):
            for i in range(self.M)[2:]:
                self.grid[i,j] = self.a[i]*self.grid[i-1,j+1] +\
                self.b[i]*self.grid[i,j+1] + \
                self.c[i]*self.grid[i+1,j+1]
                

In [4]:
class FDImplicitEu(FiniteDifferences):
    def _setup_coefficients_(self):
        self.a = 0.5*(self.r*self.dt*self.i_values -
                      (self.sigma**2)*self.dt*(self.i_values**2))
        self.b = 1 + \
                 (self.sigma**2)*self.dt*(self.i_values**2) + \
                 self.r*self.dt
        self.c = -0.5*(self.r * self.dt*self.i_values +
                       (self.sigma**2)*self.dt*(self.i_values**2))
        self.coeffs = np.diag(self.a[2:self.M], -1) + \
                      np.diag(self.b[1:self.M]) + \
                      np.diag(self.c[1:self.M-1], 1)
    
    def _traverse_grid_(self):
        """ Solve using linear systems of equations """
        P, L, U = linalg.lu(self.coeffs)
        aux = np.zeros(self.M-1)
        for j in reversed(range(self.N)):
            aux[0] = np.dot(-self.a[1], self.grid[0, j])
            x1 = linalg.solve(L, self.grid[1:self.M, j+1]+aux)
            x2 = linalg.solve(U, x1)
            self.grid[1:self.M, j] = x2
            

In [5]:
class FDCnEu(FiniteDifferences):
    def _setup_coefficients_(self):
        self.alpha = 0.25*self.dt*(
            (self.sigma**2)*(self.i_values**2) -
            self.r*self.i_values)
        
        self.beta = -self.dt*0.5*(
            (self.sigma**2)*(self.i_values**2) +
            self.r)         
        
        self.gamma = 0.25*self.dt*(
            (self.sigma**2)*(self.i_values**2) +
            self.r*self.i_values)
        
        self.M1 = -np.diag(self.alpha[2:self.M], -1) + \
            np.diag(1-self.beta[1:self.M]) - \
            np.diag(self.gamma[1:self.M-1], 1)
            
        self.M2 = np.diag(self.alpha[2:self.M], -1) + \
            np.diag(1+self.beta[1:self.M]) + \
            np.diag(self.gamma[1:self.M-1], 1)
            
    def _traverse_grid_(self):
        """ Solve using linear systems of equations """
        P, L, U = linalg.lu(self.M1)
        for j in reversed(range(self.N)):
            x1 = linalg.solve(L,
                              np.dot(self.M2,
                                     self.grid[1:self.M, j+1]))
            x2 = linalg.solve(U, x1)
            self.grid[1:self.M, j] = x2
                

In [6]:
def N(x):
    return stats.norm.cdf(x, 0.0, 1.0)

def bsm_d1(S, K, T, r, sigma):
    S = float(S)
    return (m.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * m.sqrt(T))

def bsm_d2(S, K, T, r, sigma):
    S = float(S)
    return (m.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * m.sqrt(T))

def bsm_call_pv(S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    d2 = bsm_d2(S, K, T, r, sigma)
    return S * N(d1) - K * m.exp(-r * T) * N(d2)

def bsm_call_delta(S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    return N(d1)

S0 = 80.; K = 85.; T = 1.; r = 0.05; 
sigma = 0.2

Smax = 160.

ref_pv = bsm_call_pv(S0, K, T, r, sigma)

print( "ref_pv: %.6f " % (ref_pv) )

ref_pv: 5.988244 


In [7]:
start = default_timer()

pv = FDExplicitEu(S0=S0, K=K, r=r, T=T, sigma=sigma, Smax=Smax,
                  M=100, N=1000, is_call=True).price()

calcTime = default_timer() - start
print( "PV: %.5f, abs diff: %.5f, rel diff:  %.5f" % (pv, ref_pv - pv, (ref_pv - pv)/ref_pv) )
print( "Calculation time   %.5f" % calcTime )

PV: 5.98385, abs diff: 0.00440, rel diff:  0.00073
Calculation time   0.53855


In [8]:
start = default_timer()

pv = FDExplicitEu(S0=S0, K=K, r=r, T=T, sigma=sigma, Smax=Smax,
                  M=100, N=100, is_call=True).price()

calcTime = default_timer() - start
print( "PV: %.5f, abs diff: %.5f, rel diff:  %.5f" % (pv, ref_pv - pv, (ref_pv - pv)/ref_pv) )
print( "Calculation time   %.5f" % calcTime )

PV: 12571236732673735775324723331838063331810213888.00000, abs diff: -12571236732673735775324723331838063331810213888.00000, rel diff:  -2099319459762903469099949937327727273377267712.00000
Calculation time   0.06131


In [9]:
start = default_timer()

pv = FDImplicitEu(S0=S0, K=K, r=r, T=T, sigma=sigma, Smax=Smax,
                  M=100, N=100, is_call=True).price()

calcTime = default_timer() - start
print( "PV: %.5f, abs diff: %.5f, rel diff:  %.5f" % (pv, ref_pv - pv, (ref_pv - pv)/ref_pv) )
print( "Calculation time   %.5f" % calcTime )

PV: 5.90271, abs diff: 0.08553, rel diff:  0.01428
Calculation time   0.20542


In [10]:
start = default_timer()

pv = FDImplicitEu(S0=S0, K=K, r=r, T=T, sigma=sigma, Smax=Smax,
                  M=100, N=1000, is_call=True).price()

calcTime = default_timer() - start
print( "PV: %.5f, abs diff: %.5f, rel diff:  %.5f" % (pv, ref_pv - pv, (ref_pv - pv)/ref_pv) )
print( "Calculation time   %.5f" % calcTime )

PV: 5.91710, abs diff: 0.07114, rel diff:  0.01188
Calculation time   1.04794


In [11]:
start = default_timer()

pv = FDCnEu(S0=S0, K=K, r=r, T=T, sigma=sigma, Smax=Smax,
                  M=100, N=100, is_call=True).price()

calcTime = default_timer() - start
print( "PV: %.5f, abs diff: %.5f, rel diff:  %.5f" % (pv, ref_pv - pv, (ref_pv - pv)/ref_pv) )
print( "Calculation time   %.5f" % calcTime )

PV: 5.91868, abs diff: 0.06957, rel diff:  0.01162
Calculation time   0.15963


In [12]:
start = default_timer()

pv = FDCnEu(S0=S0, K=K, r=r, T=T, sigma=sigma, Smax=Smax,
                  M=100, N=1000, is_call=True).price()

calcTime = default_timer() - start
print( "PV: %.5f, abs diff: %.5f, rel diff:  %.5f" % (pv, ref_pv - pv, (ref_pv - pv)/ref_pv) )
print( "Calculation time   %.5f" % calcTime )

PV: 5.91870, abs diff: 0.06955, rel diff:  0.01161
Calculation time   1.02299
